In [1]:
import psutil

In [2]:
for proc in psutil.process_iter():
    try:
        pinfo = proc.as_dict(attrs=['pid', 'name', 'username'])
    except psutil.NoSuchProcess:
        pass
    else:
        if 'python' in pinfo['name']:
            print(pinfo)

{'name': 'python3.6', 'pid': 31772, 'username': 'yoavfreund'}
{'name': 'python3.6', 'pid': 61039, 'username': 'yoavfreund'}
{'name': 'python3.6', 'pid': 66583, 'username': 'yoavfreund'}
{'name': 'python3.6', 'pid': 66600, 'username': 'yoavfreund'}


In [3]:
a=psutil.cpu_percent(percpu=True)
a, sum(a)

([33.1, 1.7, 30.8, 2.1, 32.1, 1.7, 32.5, 1.9], 135.9)

In [4]:
!ls ../../data/tifTiles/tiles_111*

../../data/tifTiles/tiles_111.tif  ../../data/tifTiles/tiles_1115.tif
../../data/tifTiles/tiles_1110.tif ../../data/tifTiles/tiles_1116.tif
../../data/tifTiles/tiles_1111.tif ../../data/tifTiles/tiles_1117.tif
../../data/tifTiles/tiles_1112.tif ../../data/tifTiles/tiles_1118.tif
../../data/tifTiles/tiles_1113.tif ../../data/tifTiles/tiles_1119.tif
../../data/tifTiles/tiles_1114.tif


In [5]:
from os import getpid,system
from os.path import isfile
from glob import glob

In [6]:
def getLock(lockfile):
    try:
        l=open(lockfile, 'x')
        l.write(str(getpid()))
        l.close()
        return True
    except FileExistsError:
        return False

In [155]:
from time import sleep
from os.path import isfile

In [169]:
%%writefile ProcessTiles.py
from os import getpid,system
from os.path import isfile
from glob import glob
from time import sleep
from os.path import isfile
import psutil

tile_pattern='../../data/tifTiles/tiles_*.tif'
max_load=700.
i=0
for infile in glob(tile_pattern):
    stem=infile[:-4]
    lockfile=stem+'.lock'
    if not isfile(lockfile):
        i+=1
        print('got lock',lockfile)
        command='python3 run_job.py %s &'%stem
        print(i,command,'\n')
        system(command)
        sleep(0.1)
    else:
        print('\r %s exists'%lockfile,end='')
        continue

    # Wait if load is too high
    load=sum(psutil.cpu_percent(percpu=True))
    print('\r %5d                            load: %6.2f'%(i,load))
    j=0
    while load>max_load:
        print('\r %5d    Sleep:%3d               load: %6.2f'%(i,j,load))
        j+=1
        sleep(2)
        load=sum(psutil.cpu_percent(percpu=True))
    print('\nload low enough',load)

Overwriting ProcessTiles.py


In [170]:
!python3 ProcessTiles.py

 ../../data/tifTiles/tiles_697.lock existsgot lock ../../data/tifTiles/tiles_683.lock
1 python3 run_job.py ../../data/tifTiles/tiles_683 & 



     1                            load: 278.70

load low enough 278.7
got lock ../../data/tifTiles/tiles_56.lock
2 python3 run_job.py ../../data/tifTiles/tiles_56 & 

     2                            load: 387.30

load low enough 387.3
got lock ../../data/tifTiles/tiles_873.lock
3 python3 run_job.py ../../data/tifTiles/tiles_873 & 

     3                            load: 455.30

load low enough 455.29999999999995
got lock ../../data/tifTiles/tiles_867.lock
4 python3 run_job.py ../../data/tifTiles/tiles_867 & 

     4                            load: 574.00

load low enough 574.0
got lock ../../data/tifTiles/tiles_871.lock
5 python3 run_job.py ../../data/tifTiles/tiles_871 & 

     5                            load: 752.70
     5    Sleep:  0               load: 752.70

load low enough 630.0
got lock ../../data/tifTiles/tiles_68.lock
6 python3 run_job.py ../../data/tifTiles/tiles_68 & 

     6                            load: 311.30

load low enough 311.3
got lock ../../data/tifTiles/

    42                            load: 658.10

load low enough 658.1
got lock ../../data/tifTiles/tiles_737.lock
43 python3 run_job.py ../../data/tifTiles/tiles_737 & 

    43                            load: 659.00

load low enough 659.0
got lock ../../data/tifTiles/tiles_723.lock
44 python3 run_job.py ../../data/tifTiles/tiles_723 & 

    44                            load: 746.20
    44    Sleep:  0               load: 746.20

load low enough 644.4
got lock ../../data/tifTiles/tiles_938.lock
45 python3 run_job.py ../../data/tifTiles/tiles_938 & 

    45                            load: 754.40
    45    Sleep:  0               load: 754.40

load low enough 504.6000000000001
got lock ../../data/tifTiles/tiles_910.lock
46 python3 run_job.py ../../data/tifTiles/tiles_910 & 

    46                            load: 509.80

load low enough 509.79999999999995
got lock ../../data/tifTiles/tiles_904.lock
47 python3 run_job.py ../../data/tifTiles/tiles_904 & 

    47                         

In [150]:
%%writefile run_job.py
from sys import argv
from os import getpid,system
from os.path import isfile
from time import sleep
import datetime


def getLock(lockfile):
    try:
        l=open(lockfile, 'x')
        return l
    except FileExistsError:
        return None

stem=argv[1]
lockfilename=stem+'.lock'
logfilename=stem+'.log'

lockfile=getLock(lockfilename)
if not lockfile is None:

    command='python3 ../scripts/extractPatches.py %s > %s'%(stem,logfilename)
    # put some info into the log/lock file
    now = datetime.datetime.now()
    print('pid=',getpid(), file=lockfile)
    print('start time=',now.isoformat(), file=lockfile)
    print('command=',command, file=lockfile)
    
    system(command)
    sleep(1)
else:
    print(lockfilename,'exists, skipping')


Overwriting run_job.py


In [137]:
%pwd

'/Users/yoavfreund/projects/shapeology/shapeology_code/notebooks'

In [151]:
stem='/Users/yoavfreund/projects/shapeology/data/tifTiles/tiles_00'
!rm {stem}.l*
!ls {stem}*

/Users/yoavfreund/projects/shapeology/data/tifTiles/tiles_00.tif


In [147]:
!python3 ../scripts/extractPatches.py /Users/yoavfreund/projects/shapeology/data/tifTiles/tiles_00 > /Users/yoavfreund/projects/shapeology/data/tifTiles/tiles_00.log

In [152]:
!python3 run_job.py $stem

In [153]:
!cat {stem}.lo*

pid= 33233
start time= 2018-09-08T10:10:40.730924
command= python3 ../scripts/extractPatches.py /Users/yoavfreund/projects/shapeology/data/tifTiles/tiles_00 > /Users/yoavfreund/projects/shapeology/data/tifTiles/tiles_00.log
image /Users/yoavfreund/projects/shapeology/data/tifTiles/tiles_00.tif too blank, skipping


In [154]:
!ls -l {stem}.*

-rw-r--r--  1 yoavfreund  staff      224 Sep  8 10:10 /Users/yoavfreund/projects/shapeology/data/tifTiles/tiles_00.lock
-rw-r--r--  1 yoavfreund  staff       91 Sep  8 10:10 /Users/yoavfreund/projects/shapeology/data/tifTiles/tiles_00.log
-rw-r--r--  1 yoavfreund  staff  6008316 Sep  6 20:49 /Users/yoavfreund/projects/shapeology/data/tifTiles/tiles_00.tif


In [42]:
import datetime

now = datetime.datetime.now()
print(now.isoformat())
print(now.strftime("%Y-%m-%d %H:%M"))

2018-09-08T09:27:21.653231
2018-09-08 09:27


In [12]:
!cat ../../data/tifTiles/tiles_*.log

image ../../data/tifTiles/tiles_00.tif too blank, skipping
image ../../data/tifTiles/tiles_01.tif too blank, skipping
image ../../data/tifTiles/tiles_02.tif too blank, skipping
image ../../data/tifTiles/tiles_03.tif too blank, skipping
image ../../data/tifTiles/tiles_07.tif too blank, skipping
image ../../data/tifTiles/tiles_1058.tif too blank, skipping
image ../../data/tifTiles/tiles_1059.tif too blank, skipping
image ../../data/tifTiles/tiles_1063.tif too blank, skipping
image ../../data/tifTiles/tiles_1064.tif too blank, skipping
image ../../data/tifTiles/tiles_1065.tif too blank, skipping
processing ../../data/tifTiles/tiles_1066.tif into ../../data/tifTiles/tiles_1066_extracted.pkl
found 143 patches
finished in  32.1 seconds
image ../../data/tifTiles/tiles_1067.tif too blank, skipping
image ../../data/tifTiles/tiles_1070.tif too blank, skipping
image ../../data/tifTiles/tiles_1071.tif too blank, skipping
image ../../data/tifTiles/tiles_1072.tif too blank, skipping
image ../../data

In [78]:
!rm ../../data/tifTiles/tiles_*.lock

In [40]:
!rm ../../data/tifTiles/tiles_29.lock

In [42]:
!cat ../../data/tifTiles/tiles_29.lock

72484